# Childes Preprocsssing 

In [1]:
def listify_data(raw_string):
    return [line.split() for line in raw_string.splitlines()]

def read_data(filename):
    with open(filename, "r") as f:
        raw_data = f.read()
        data = listify_data(raw_data)
    return data

train = read_data("childes-txt/train.txt")
valid = read_data("childes-txt/valid.txt")
test = read_data("childes-txt/test.txt")
excluded = read_data("childes-txt/excluded.txt")
treebank = read_data("childes-txt/treebank.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'childes-txt/excluded.txt'

In [ ]:
def unlistify_data(data):
    zipped = [" ".join(line) for line in data]
    return "\n".join(zipped)

def write_data(data, filename):
    with open(filename, "w") as f:
        f.write(unlistify_data(data))

## Split possesives and contractions

Seperate possesives: "camel's" -> "camel 's"

Also seperate contractions. The subwords _n't_ , _'re_ , _'ll_ , _'m_ , _'ve_ , and _'d_ should be prepened with spaces. 

In [ ]:
# this takes a minute and a half to run on my machine

def split_possesives_and_contractions(word):
    if word.endswith("'s"):
        return word[:-2] + " 's"
    if word == "can't":
        return "can n't"
    if word.endswith("n't"):
        return word[:-3] + " n't"
    if word.endswith("'re"):
        return word[:-3] + " 're"
    if word.endswith("'m"):
        return word[:-2] + " 'm"
    if word.endswith("'d"):
        return word[:-2] + " 'd"
    if word.endswith("'ll"):
        return word[:-3] + " 'll"
    if word.endswith("'ve"):
        return word[:-3] + " 've"
    if word.endswith("s'"):
        return word[:-1] + " 's'"
    if word.endswith("'r"):
        return word[:-2] + " are"
    if word.endswith("'has"):
        return word[:-4] + " has"
    if word.endswith("'is"):
        return word[:-3] + " is"
    if word.endswith("'did"):
        return word[:-4] + " did"
    if word == "y'all":
        return "you all"
    if word == "c'mere":
        return "come here"
    if word == "I'ma":
        return "I am going to"
    if word == "what'cha":
        return "what are you"
    if word == "don'tcha":
        return "do you not"
    
    # List of startswith exceptions: ["t'", "o'", "O'", "d'"]
    # List of == exceptions: ["Ma'am", "ma'am", "An'", "b'ring", "Hawai'i","don'ting", "rock'n'roll" "don'ting", "That'scop","that'ss","go'ed", "s'pose", "'hey", "me'", "shh'ell", "th'do", "Ross'a", "him'sed"] 
    # List of in exceptions: ["_", "-"]
    # List of endswith exceptions (note that this one is a catch all condition): ["'"]

    return word

def split_line(line):
    s = [split_possesives_and_contractions(word) for word in line]
    return " ".join(s).split()

def split_data(data):
    return [split_line(line) for line in data]

train_split = split_data(train)
valid_split = split_data(valid)
test_split = split_data(test)
write_data(split_data(excluded), "excluded-processed.txt")
write_data(split_data(excluded), "treebank-processed.txt")

## Unking

Replace infrequent words with `<unk>` tokens. 

Note that the unked tokens are based on the training set, even for the validation and test sets.

In [ ]:
def count_frequencies(data):
    frequencies = {}
    for line in data:
        for word in line:
            if word in frequencies:
                frequencies[word] += 1
            else:
                frequencies[word] = 1
    return frequencies

# words with frequency > cutoff
def make_vocab(data, cutoff):
    frequencies = count_frequencies(data)
    high_frequency_tokens = set()
    for token in frequencies:
        if frequencies[token] > cutoff:
            high_frequency_tokens.add(token)
    return high_frequency_tokens

def unk(data, vocab):
    unked_data = []
    for line in data:
        unked_line = []
        for word in line:
            if word in vocab:
                unked_line.append(word)
            else:
                unked_line.append("<unk>")
        unked_data.append(unked_line) 
    return unked_data

# train vocab used for train, valid, and test sets
train_vocab = make_vocab(train_split, cutoff=2)
train_unked = unk(train_split, train_vocab)
valid_unked = unk(valid_split, train_vocab)
test_unked  = unk(test_split,  train_vocab)

In [ ]:
with open ("childes-txt-unked/vocab.txt", 'w') as f:
    f.write("\n".join(train_vocab))
write_data(train_unked, "childes-txt-unked/train.txt")
write_data(valid_unked, "childes-txt-unked/valid.txt")
write_data(test_unked, "childes-txt-unked/test.txt")